<a href="https://colab.research.google.com/github/getaccept/notebooks/blob/master/API_Export_document_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Export document data

Functions to export document data as CSV file

In [ ]:
# import dependencies
import requests
import os
import io
import csv
from google.colab import files

In [ ]:
#@title Enter GetAccept login information
#@markdown Remember to keep your credentials safe
EMAIL = "" #@param {type:"string"}
PASSWORD = "" #@param {type:"string"}
BASEURL = "https://api.getaccept.com/v1" #@param {type:"string"}
ACTION = "Signed documents" #@param ["Signed documents","All documents"]
SOURCE_ENTITY = "" #@param {type: "string"}
LAST_DOCUMENTS_COUNT = 500 #@param {type: "number"}
EXPORT_FILENAME = "export.csv" #@param {type: "string"}

In [ ]:
# Make auth request and store auth header
if EMAIL and PASSWORD:
  payload = { "email": EMAIL, "password": PASSWORD, "entity_id": SOURCE_ENTITY }
  response = requests.post(BASEURL+"/auth", json=payload)
  data = response.json()
  if "access_token" in data:
    source_auth_headers = { "Authorization": "bearer " + data["access_token"]}
  else:
    raise TypeError(data["errors"], "Please check your credentials")
  # Check login and list entities
  response = requests.get(BASEURL+"/users/me", headers=source_auth_headers)
  user_data = response.json()
  print("Logged in as " + user_data["user"]["first_name"] + " on entity " + user_data["user"]["entity_name"])
  if len(user_data["entities"]) > 1:
    print("Available entities:\r\n-", '\r\n- '.join(map(lambda x: x["name"]+": \t"+x["id"], user_data["entities"])))


In [ ]:
if ACTION == "All documents":
  # Get data for all documents
  response = requests.get(BASEURL+"/documents?showall=true&sort_by=created&sort_order=desc&offset=0&limit="+str(LAST_DOCUMENTS_COUNT),headers=source_auth_headers)
  document_data = response.json()
if ACTION == "Signed documents":
  # Get data for signed documents
  response = requests.get(BASEURL+"/documents?filter=signed&showall=true&sort_order=desc&offset=0&limit="+str(LAST_DOCUMENTS_COUNT),headers=source_auth_headers)
  document_data = response.json()


In [ ]:
# Get field names and flatten recipients
fieldnames = []
for document in document_data:
  for key, field in enumerate(document):
    if field not in fieldnames and not isinstance(document[field],list):
      fieldnames.append(field)

  if 'recipients' in document:
    for recid,recipient in enumerate(document['recipients']):
      for key, value in recipient.items():
        fieldname = 'recipient{}_{}'.format(recid+1,key)
        document[fieldname] = value
        if not fieldname in fieldnames:
          fieldnames.append(fieldname)
  
    del document['recipients']

In [ ]:
# Export and download CSV file with data
with open(EXPORT_FILENAME, 'w', newline='') as csv_file:
    # Attach a CSV writer to the file with the desired fieldnames
    writer = csv.DictWriter(csv_file, fieldnames)
    # Write the header row
    writer.writeheader()

    # Write document data to rows
    for row in document_data:
      writer.writerow(row)
files.download(EXPORT_FILENAME) 
